In [41]:
import sys
import time
import logging
import os
import opcua

In [42]:
# url = "opc.tcp://130.159.75.138:48050"
# url = "opc.tcp://CX-696B93:4840"
url = "opc.tcp://130.159.75.138:4840"

In [43]:
client = opcua.Client(url = url)
client.set_user(username = 'Administrator')
client.set_password(pwd = '*500TPress*')

In [44]:
client.connect()
objects = client.get_objects_node()
childs=objects.get_children()

In [51]:
p = client.get_node("ns=4;s=GVL_Analog_Engineering_Units.lrEL3202_1_Ambient_Temperature_Front")
var = p.get_data_value()
var.Value.Value

19.9

In [49]:
var = p.get_data_value()

In [50]:
var.Value.Value

19.8

In [6]:
objects.get_children()a

[Node(FourByteNodeId(i=2253)),
 Node(StringNodeId(ns=1;s=TcOpcUaServerRoot)),
 Node(StringNodeId(ns=1;s=License)),
 Node(FourByteNodeId(ns=2;i=5001))]

In [7]:
bulb = objects.get_children()[1]
val = bulb.get_browse_name()
val

QualifiedName(1:TcOpcUaServer)

In [131]:
val.Name

'TcOpcUaServer'

In [ ]:
val.

In [113]:
bulb.get_children()[1]

IndexError: list index out of range

In [ ]:
opc.cp://CX-696B93:48040

In [64]:
root = client.get_root_node()
root

Node(TwoByteNodeId(i=84))

In [65]:
objects = client.get_objects_node()
objects

Node(TwoByteNodeId(i=85))

In [8]:
client.activate_session(username="Administrator", password="*500TPress*", certificate=None)

ServiceFault from server received  in response to ActivateSessionRequest


BadIdentityTokenInvalid: "The user identity token is not valid."(BadIdentityTokenInvalid)

In [62]:
client.activate_session(username="Administrator", password="*500TPress*", certificate=None)

AttributeError: 'NoneType' object has no attribute 'send_request'

In [25]:
client = opcua.Client("http://130.159.75.138:48050")

# 48050 - UA gateway (middleman with actual PLC)

In [60]:
client.connect()

ServiceFault from server received  in response to ActivateSessionRequest


BadIdentityTokenRejected: "The user identity token is valid but the server has rejected it."(BadIdentityTokenRejected)

In [27]:
client.get_namespace_array()

['http://opcfoundation.org/UA/',
 'urn:BeckhoffAutomation:TcOpcUaGateway',
 'http://opcfoundation.org/UA/DI/',
 'http://unifiedautomation.com/Configuration/',
 'http://unifiedautomation.com/TagFile/']

In [28]:
client.load_type_definitions() 

([], {})

Node(TwoByteNodeId(i=84))

In [31]:
objects = client.get_objects_node()
objects

Node(TwoByteNodeId(i=85))

In [ ]:
client.load_type_definitions()  # load definition of server specific structures/extension objects

# Client has a few methods to get proxy to UA nodes that should always be in address space such as Root or Objects
root = client.get_root_node()
print("Root node is: ", root)
objects = client.get_objects_node()
print("Objects node is: ", objects)